In [73]:
import tensorflow as tf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input,Dense, Subtract, Add, Concatenate
from tensorflow.keras.optimizers import Adam

real_dem = pd.read_excel('C:/Users/ERIC/Desktop/Forecast Real Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',0)
real_dem_daybef = pd.read_excel('C:/Users/ERIC/Desktop/Forecast Real Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',1)
forec_dem = pd.read_excel('C:/Users/ERIC/Desktop/Forecast Real Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',2)
# Dataframes
real_dem = pd.DataFrame(real_dem)
real_dem_daybef = pd.DataFrame(real_dem_daybef)
forec_dem = pd.DataFrame(forec_dem)
#Data from the day before
cen_x_train_daybef = real_dem_daybef.iloc[:20,7:8]
cen_x_train_daybef = pd.DataFrame(cen_x_train_daybef)

#Data from the forecast
cen_x_train_forec = forec_dem.iloc[:20,3:4]
cen_x_train_forec = pd.DataFrame(cen_x_train_forec)

# Training dataset
cen_x_train = real_dem.iloc[0:20, 3:7].values
cen_x_train = pd.DataFrame(cen_x_train)

cen_y_train = real_dem.iloc[0:20, 7:8].values
cen_y_train = pd.DataFrame(cen_y_train)


# Test dataset
cen_x_test = real_dem.iloc[20:24, 3:7].values
cen_x_test = pd.DataFrame(cen_x_test)

cen_y_test = real_dem.iloc[20:24, 7:8].values
cen_y_test = pd.DataFrame(cen_y_test)

# Input from the current day
input_tensor1 = Input(shape=(1,))
input_tensor2 = Input(shape=(1,))
input_tensor3 = Input(shape=(1,))
input_tensor4 = Input(shape=(1,))
# Input from the current day
input_tensor5 = Input(shape=(1,))
# Input from the forecast
input_tensor6 = Input(shape=(1,))

shared_layer = Dense(1)

tensor_shared1 = shared_layer(input_tensor2)
tensor_shared2 = shared_layer(input_tensor3)

output_tensor = Concatenate()([input_tensor1, tensor_shared1,tensor_shared2, input_tensor4])

hidden_layer = Dense(50)(output_tensor)
hidden_layer2 = Dense(50)(input_tensor5)
hidden_layer3 = Dense(50)(input_tensor6)

shared_layer2 = Dense(1)
shared_hidden_layer = shared_layer2(hidden_layer2)
shared_hidden_layer2 = shared_layer2(hidden_layer3)

out_tensor =  Concatenate()([hidden_layer,shared_hidden_layer,shared_hidden_layer2])
hidden_layer4 = Dense(50)(out_tensor)

out_tensor = Dense(24)(hidden_layer4)

#output_layer = Dense(4)(diff_dem3)
model = Model([input_tensor1,input_tensor2,input_tensor3,input_tensor4, input_tensor5,input_tensor6], out_tensor)

model.compile(optimizer= Adam(0.01),  loss='mean_absolute_error', metrics = ['accuracy'])

model.summary()

model.fit([cen_x_train[0],cen_x_train[1],cen_x_train[2],cen_x_train[3],
          cen_x_train_daybef, cen_x_train_forec],
          cen_y_train,
          epochs=20,
          batch_size=2,
          validation_split=0.1,
          verbose=True)



Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_214 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_215 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_213 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_116 (Dense)               (None, 1)            2           input_214[0][0]                  
                                                                 input_215[0][0]           